## Adding:
- stellar mass via abundance matching
- r$\rm_{vir}$
- redshift

In [1]:
import numpy as np
import pandas as pd
from cosmologicalTools import CosmologicalTools as cosmo 
from abundanceMatchingClass import AM

In [2]:
snapData = np.genfromtxt('data_spacing.txt')
snaps = snapData[:,0]
scale = snapData[:,1]
redshift = snapData[:,2]


In [3]:
df = pd.read_csv('data_dwarfGroupSubhalos_84-initial.csv')
z = 1.0355
a = 1/(1+z)
df[0:10]
cosmology = cosmo( OmegaM=0.2726, OmegaL= 0.7274, OmegaR=0, h=0.704)
mass = df['Mass at z=0']
maxMass = df['Max Mass']
maxMassTrue = maxMass*1e10
maxMassSnap = df['Max Mass Snap']
maxMassRedshifts = [redshift[snaps == i][0] for i in maxMassSnap]
maxMassStellar = [AM(maxMassTrue[i], maxMassRedshifts[i]).stellarMass()/1e10 for i in range(len(maxMassTrue))]
rvirMax = [cosmology.rvir(maxMassRedshifts[i],maxMass[i]) for i in range(len(maxMassRedshifts))]
rvir = [cosmology.rvir(z,mass[i]) for i in range(len(mass))]

df.insert(5,'Redshift',maxMassRedshifts)
df.insert(13, 'AM Stellar Mass', maxMassStellar)
df.insert(6, 'Rvir at z=0',rvir)
df.insert(7, 'Rvir at Max Mass',rvirMax)
df.drop(['Flag'],axis = 1, inplace = True)
df.to_csv('data_dwarfGroupSubhalos_84.csv',index=False,header=True)
df[0:10]

,Group Number,Subhalo ID,Mass at z=0,Max Mass,Max Mass Snap,Redshift,Rvir at z=0,Rvir at Max Mass,Pos x,Pos y,Pos z,Vel x,Vel y,Vel z,AM Stellar Mass
0,1432,365764,113.775405,117.274837,76,1.4141,232.257152,230.776373,40764.039062,74174.593750,43333.339844,-120.353661,-393.236206,16.545963,2.160810
1,1432,365765,0.673112,1.198063,69,1.9041,42.006386,49.512682,40679.050781,74249.234375,43229.480469,104.159531,-397.724731,-115.497261,0.000551
2,1432,365766,0.512166,0.815255,78,1.3024,38.349232,44.216135,40901.054688,74095.578125,43305.890625,-364.170624,-308.107635,-49.979893,0.000250
3,1432,365767,0.497877,0.866396,77,1.3576,37.989213,45.031443,40797.421875,74231.226562,43184.039062,12.781004,-452.561340,-203.141357,0.000283
4,1432,365768,0.297072,0.594143,68,2.0020,31.982077,39.132834,41084.242188,74260.773438,43221.261719,-113.886513,-481.217072,9.177190,0.000154
5,1432,365769,0.253451,0.373032,64,2.4442,30.333157,33.346010,40744.988281,74037.984375,43332.285156,89.622887,-527.347900,78.292763,0.000071
6,1432,365770,0.249691,0.311361,75,1.4720,30.182393,31.898659,40821.835938,74210.703125,43548.171875,-110.178436,-199.096161,-192.136719,0.000042
7,1432,365771,0.236905,0.279774,76,1.4141,29.658179,30.836048,40937.187500,74097.320312,43351.796875,-200.845383,-554.336914,228.326843,0.000034
8,1432,365772,0.212839,0.251195,69,1.9041,28.617813,29.414401,40671.207031,74278.460938,43171.976562,-17.203236,-118.643204,44.140270,0.000031
9,1432,365773,0.198549,0.357990,69,1.9041,27.962475,33.101437,40877.886719,74002.203125,43470.476562,38.498703,-417.634125,30.770889,0.000060


___

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from positionCorrection import positionCorrection as correct
from mpl_toolkits.mplot3d import Axes3D
import math

In [5]:
df = pd.read_csv('data_dwarfGroupSubhalos_84.csv')
df[0:10]

,Group Number,Subhalo ID,Mass at z=0,Max Mass,Max Mass Snap,Redshift,Rvir at z=0,Rvir at Max Mass,Pos x,Pos y,Pos z,Vel x,Vel y,Vel z,AM Stellar Mass
0,1432,365764,113.775405,117.274837,76,1.4141,232.257152,230.776373,40764.039062,74174.593750,43333.339844,-120.353661,-393.236206,16.545963,2.160810
1,1432,365765,0.673112,1.198063,69,1.9041,42.006386,49.512682,40679.050781,74249.234375,43229.480469,104.159531,-397.724731,-115.497261,0.000551
2,1432,365766,0.512166,0.815255,78,1.3024,38.349232,44.216135,40901.054688,74095.578125,43305.890625,-364.170624,-308.107635,-49.979893,0.000250
3,1432,365767,0.497877,0.866396,77,1.3576,37.989213,45.031443,40797.421875,74231.226562,43184.039062,12.781004,-452.561340,-203.141357,0.000283
4,1432,365768,0.297072,0.594143,68,2.0020,31.982077,39.132834,41084.242188,74260.773438,43221.261719,-113.886513,-481.217072,9.177190,0.000154
5,1432,365769,0.253451,0.373032,64,2.4442,30.333157,33.346010,40744.988281,74037.984375,43332.285156,89.622887,-527.347900,78.292763,0.000071
6,1432,365770,0.249691,0.311361,75,1.4720,30.182393,31.898659,40821.835938,74210.703125,43548.171875,-110.178436,-199.096161,-192.136719,0.000042
7,1432,365771,0.236905,0.279774,76,1.4141,29.658179,30.836048,40937.187500,74097.320312,43351.796875,-200.845383,-554.336914,228.326843,0.000034
8,1432,365772,0.212839,0.251195,69,1.9041,28.617813,29.414401,40671.207031,74278.460938,43171.976562,-17.203236,-118.643204,44.140270,0.000031
9,1432,365773,0.198549,0.357990,69,1.9041,27.962475,33.101437,40877.886719,74002.203125,43470.476562,38.498703,-417.634125,30.770889,0.000060


In [6]:
groups      = df['Group Number']
subID       = df['Subhalo ID']
mass        = df['Mass at z=0']
maxMass     = df['Max Mass']
maxMassSnap = df['Max Mass Snap']
redshift    = df['Redshift']
rvir        = df['Rvir at z=0']
rvirMaxMass = df['Rvir at Max Mass']
position    = np.column_stack([df['Pos x'],df['Pos y'],df['Pos z']])
velocity    = np.column_stack([df['Vel x'],df['Vel y'],df['Vel z']])
stellarMass = df['AM Stellar Mass']

In [7]:
uniqueGroups = np.unique(groups)
print(len(uniqueGroups),uniqueGroups[0])

# going to ignore groups that have subhalos with 3.2e11 Msun at z=0
uniqueGroups = [i for i in uniqueGroups if all(mass[groups == i]<32)==True] 
print(len(uniqueGroups), uniqueGroups[0])

#uniqueGroups = uniqueGroups[0:50] # for testing only

13083 1432
12806 2703


in order to get a full row out of a data frame:

$\rm df.iloc[row,:]$

to get that row as a list:

$\rm df.iloc[row,:].values[0]$

or an array:

$\rm np.array(df.iloc[row,:].values[0])$

In [8]:
primaries = []
secondaries = []
cols = ['Group Number', 'Subhalo ID', 'Mass at z=0', 'Max Mass',
       'Max Mass Snap', 'Redshift', 'Rvir at z=0', 'Rvir at Max Mass', 'Pos x',
       'Pos y', 'Pos z', 'Vel x', 'Vel y', 'Vel z', 'AM Stellar Mass']

positionInDF = np.arange(0,len(df))
numberOfSingleSubhaloGroups = 0

for i in uniqueGroups:
    groupMask = groups == i
    masses = mass[groupMask]
    if len(masses)>1:
        firstMassive = np.sort(masses)[-1]
        secondMassive = np.sort(masses)[-2]

        locationOfPrimary = firstMassive == mass[groupMask]
        locationOfSecondary = secondMassive == mass[groupMask]
        
        primaryInfo = df.iloc[positionInDF[groupMask][locationOfPrimary],:].values[0]
        primaries.append(primaryInfo)
        
        secondaryInfo = df.iloc[positionInDF[groupMask][locationOfSecondary],:].values[0]
        secondaries.append(secondaryInfo)
    else:
        numberOfSingleSubhaloGroups = numberOfSingleSubhaloGroups+1
        
        
prims = pd.DataFrame(data = primaries, columns=cols)
prims.to_csv('data_primaries_84.csv',index=False,header=True)

secos = pd.DataFrame(data = secondaries, columns=cols)
secos.to_csv('data_secondaries_84.csv',index=False,header=True)

secos[0:10]

,Group Number,Subhalo ID,Mass at z=0,Max Mass,Max Mass Snap,Redshift,Rvir at z=0,Rvir at Max Mass,Pos x,Pos y,Pos z,Vel x,Vel y,Vel z,AM Stellar Mass
0,2703.0,436183.0,11.900913,13.896933,75.0,1.4720,109.432517,113.150713,39901.554688,21819.048828,15264.976562,6.604505,-23.169617,-60.322956,0.052991
1,2915.0,445467.0,8.151042,10.908920,71.0,1.7436,96.462241,103.690501,444.727386,9067.576172,32484.167969,42.124290,90.565704,-178.860718,0.032178
2,2955.0,446952.0,6.336274,11.836987,80.0,1.2063,88.694688,108.294251,35198.863281,20589.335938,38783.007812,-198.437897,-26.109936,221.710663,0.041947
3,3004.0,448744.0,5.417984,8.478197,79.0,1.2485,84.184513,96.718804,27441.402344,20027.751953,24950.753906,-334.681610,-130.694794,124.255623,0.021947
4,3012.0,449074.0,9.533365,11.463203,75.0,1.4720,101.633070,106.117329,64235.433594,38703.500000,727.602356,169.582611,-282.929871,53.319340,0.036981
5,3039.0,450095.0,18.072481,21.129687,83.0,1.0745,125.784044,132.220323,54360.640625,3153.002930,30866.017578,14.846939,-40.938656,343.123535,0.132855
6,3101.0,452426.0,22.191623,22.785013,83.0,1.0745,134.694333,135.586659,60909.597656,10154.671875,45066.613281,-44.712784,-86.009445,-361.980927,0.153267
7,3123.0,453183.0,7.016906,8.318756,82.0,1.1142,91.763123,96.703875,60455.789062,9846.987305,42095.183594,-20.363504,-173.853607,-391.000092,0.021878
8,3235.0,457352.0,7.890823,14.880654,80.0,1.2063,95.424609,116.877889,56422.683594,8674.187500,159.668777,-89.685341,13.728562,23.218424,0.064849
9,3297.0,459591.0,11.406045,15.564294,82.0,1.1142,107.894174,119.161022,31642.076172,28083.855469,24251.103516,-99.311455,-127.333122,242.482605,0.072979


In [9]:
numberOfSingleSubhaloGroups

1168

In [13]:
import numpy as np
import pandas as pd
from cosmologicalTools import CosmologicalTools as cosmo 
from densityProfiles import NFWProfile as NFW
from positionCorrection import positionCorrection as correct


In [14]:
prims = pd.read_csv('data_primaries_84.csv')
secos = pd.read_csv('data_secondaries_84.csv')
prims[0:10]

,Group Number,Subhalo ID,Mass at z=0,Max Mass,Max Mass Snap,Redshift,Rvir at z=0,Rvir at Max Mass,Pos x,Pos y,Pos z,Vel x,Vel y,Vel z,AM Stellar Mass
0,2703.0,436182.0,28.296257,28.296257,84.0,1.0355,146.059327,146.059327,40186.984375,21750.818359,15153.346680,-124.746666,-23.467604,-125.604088,0.234837
1,2915.0,445466.0,16.654811,16.654811,84.0,1.0355,122.405107,122.405107,474.626587,8948.607422,32566.701172,27.777328,128.158356,-6.720659,0.085788
2,2955.0,446951.0,21.297401,22.239757,82.0,1.1142,132.860281,134.215011,35126.203125,20557.484375,38637.812500,-191.681274,22.367399,154.107269,0.143844
3,3004.0,448743.0,31.837799,31.837799,84.0,1.0355,151.914996,151.914996,27549.230469,19854.552734,24900.384766,-283.518280,-63.079563,244.872787,0.292535
4,3012.0,449073.0,18.077746,18.077746,84.0,1.0355,125.796259,125.796259,64266.925781,38473.863281,1039.737549,117.159966,-288.106476,71.608025,0.100401
5,3039.0,450094.0,30.474281,30.474281,84.0,1.0355,149.714588,149.714588,54087.312500,3098.053223,30747.787109,96.731079,45.573910,367.529236,0.269694
6,3101.0,452425.0,27.068862,27.068862,84.0,1.0355,143.916182,143.916182,61152.304688,10161.836914,44786.535156,-172.105087,-197.377243,-275.357300,0.216104
7,3123.0,453182.0,31.234633,31.234633,84.0,1.0355,150.949531,150.949531,60343.558594,9909.165039,41818.160156,24.182831,-268.772675,-151.909195,0.282336
8,3235.0,457351.0,19.505945,19.505945,84.0,1.0355,129.025425,129.025425,56281.824219,8635.853516,81.570641,11.209549,38.127743,82.198120,0.116124
9,3297.0,459590.0,28.557981,28.557981,84.0,1.0355,146.508267,146.508267,31866.214844,28117.791016,24393.017578,-177.183548,-95.003799,126.643997,0.238918


In [15]:
primaryPos = np.column_stack([prims['Pos x'],prims['Pos x'],prims['Pos x']])
secondaryPos = np.column_stack([secos['Pos x'],secos['Pos x'],secos['Pos x']])
correctedSeparations = np.array([np.linalg.norm(i) for i in np.array(correct(primaryPos,secondaryPos))])*a

#rvirMax = [cosmology.rvir(maxMassRedshifts[i],maxMass[i]) for i in range(len(maxMassRedshifts))]
massesz0 = prims['Mass at z=0']*1e10
massesMax = prims['Max Mass']*1e10
escapez0 = [NFW(massesz0[i]).escapeVelocity(correctedSeparations[i]) for i in range(len(prims))]
escapeMax = [NFW(massesMax[i]).escapeVelocity(correctedSeparations[i]) for i in range(len(prims))]

secos.insert(15,'Escape Speed z=0',escapez0)
secos.insert(16,'Escape Speed Max Mass',escapeMax)

secos[0:10]



,Group Number,Subhalo ID,Mass at z=0,Max Mass,Max Mass Snap,Redshift,Rvir at z=0,Rvir at Max Mass,Pos x,Pos y,Pos z,Vel x,Vel y,Vel z,AM Stellar Mass,Escape Speed z=0,Escape Speed Max Mass
0,2703.0,436183.0,11.900913,13.896933,75.0,1.4720,109.432517,113.150713,39901.554688,21819.048828,15264.976562,6.604505,-23.169617,-60.322956,0.052991,178.706409,178.706409
1,2915.0,445467.0,8.151042,10.908920,71.0,1.7436,96.462241,103.690501,444.727386,9067.576172,32484.167969,42.124290,90.565704,-178.860718,0.032178,213.911064,213.911064
2,2955.0,446952.0,6.336274,11.836987,80.0,1.2063,88.694688,108.294251,35198.863281,20589.335938,38783.007812,-198.437897,-26.109936,221.710663,0.041947,210.934096,212.718266
3,3004.0,448744.0,5.417984,8.478197,79.0,1.2485,84.184513,96.718804,27441.402344,20027.751953,24950.753906,-334.681610,-130.694794,124.255623,0.021947,219.001562,219.001562
4,3012.0,449074.0,9.533365,11.463203,75.0,1.4720,101.633070,106.117329,64235.433594,38703.500000,727.602356,169.582611,-282.929871,53.319340,0.036981,216.704795,216.704795
5,3039.0,450095.0,18.072481,21.129687,83.0,1.0745,125.784044,132.220323,54360.640625,3153.002930,30866.017578,14.846939,-40.938656,343.123535,0.132855,182.714569,182.714569
6,3101.0,452426.0,22.191623,22.785013,83.0,1.0745,134.694333,135.586659,60909.597656,10154.671875,45066.613281,-44.712784,-86.009445,-361.980927,0.153267,183.676432,183.676432
7,3123.0,453183.0,7.016906,8.318756,82.0,1.1142,91.763123,96.703875,60455.789062,9846.987305,42095.183594,-20.363504,-173.853607,-391.000092,0.021878,216.738976,216.738976
8,3235.0,457352.0,7.890823,14.880654,80.0,1.2063,95.424609,116.877889,56422.683594,8674.187500,159.668777,-89.685341,13.728562,23.218424,0.064849,191.996168,191.996168
9,3297.0,459591.0,11.406045,15.564294,82.0,1.1142,107.894174,119.161022,31642.076172,28083.855469,24251.103516,-99.311455,-127.333122,242.482605,0.072979,188.404951,188.404951


In [16]:
secos.to_csv('data_secondaries_84.csv',index=False,header=True)
secos[0:10]

,Group Number,Subhalo ID,Mass at z=0,Max Mass,Max Mass Snap,Redshift,Rvir at z=0,Rvir at Max Mass,Pos x,Pos y,Pos z,Vel x,Vel y,Vel z,AM Stellar Mass,Escape Speed z=0,Escape Speed Max Mass
0,2703.0,436183.0,11.900913,13.896933,75.0,1.4720,109.432517,113.150713,39901.554688,21819.048828,15264.976562,6.604505,-23.169617,-60.322956,0.052991,178.706409,178.706409
1,2915.0,445467.0,8.151042,10.908920,71.0,1.7436,96.462241,103.690501,444.727386,9067.576172,32484.167969,42.124290,90.565704,-178.860718,0.032178,213.911064,213.911064
2,2955.0,446952.0,6.336274,11.836987,80.0,1.2063,88.694688,108.294251,35198.863281,20589.335938,38783.007812,-198.437897,-26.109936,221.710663,0.041947,210.934096,212.718266
3,3004.0,448744.0,5.417984,8.478197,79.0,1.2485,84.184513,96.718804,27441.402344,20027.751953,24950.753906,-334.681610,-130.694794,124.255623,0.021947,219.001562,219.001562
4,3012.0,449074.0,9.533365,11.463203,75.0,1.4720,101.633070,106.117329,64235.433594,38703.500000,727.602356,169.582611,-282.929871,53.319340,0.036981,216.704795,216.704795
5,3039.0,450095.0,18.072481,21.129687,83.0,1.0745,125.784044,132.220323,54360.640625,3153.002930,30866.017578,14.846939,-40.938656,343.123535,0.132855,182.714569,182.714569
6,3101.0,452426.0,22.191623,22.785013,83.0,1.0745,134.694333,135.586659,60909.597656,10154.671875,45066.613281,-44.712784,-86.009445,-361.980927,0.153267,183.676432,183.676432
7,3123.0,453183.0,7.016906,8.318756,82.0,1.1142,91.763123,96.703875,60455.789062,9846.987305,42095.183594,-20.363504,-173.853607,-391.000092,0.021878,216.738976,216.738976
8,3235.0,457352.0,7.890823,14.880654,80.0,1.2063,95.424609,116.877889,56422.683594,8674.187500,159.668777,-89.685341,13.728562,23.218424,0.064849,191.996168,191.996168
9,3297.0,459591.0,11.406045,15.564294,82.0,1.1142,107.894174,119.161022,31642.076172,28083.855469,24251.103516,-99.311455,-127.333122,242.482605,0.072979,188.404951,188.404951


___